# Redis Concurrency

Redis is implemented as a (mostly) single-threaded process, so at any given instant only one operation is being executed and all others are blocked. The resulting effect is that every Redis command is executed atomically, is consistent and isolated. It is  a TCP server that uses that client-server model and a Request/Response protocol.

Pipelining is a communication mode in which commands are buffered by the client until the application sends it for execution by the server. The commands in the pipeline are run in order, but commands from other clients may interleave them. Once the pipeline is exhausted, Redis returns all the replies to the client.

Redis' transactions allow grouping multiple commands in a single execution block. When a transaction is exectued, its constituent commands are executed in order while blocking other clients' operations.

## The `redis-server` process

The Redis server is implemented as a (mostly<sup>1</sup>) single-threaded process. In essence, it is an event loop that handles the incoming requests from clients and performs housekeeping tasks.

Redis is a TCP server that uses that client-server model and a Request/Response protocol, in which after opening a connection:

1. The client sends a request to the server, and waits for the reply (i.e. block while reading the socket)
2. The server processes the request
3. The server sends the reply back to the requesting client

A typical Request/Response server waits until the client reads the reply, before accepting another request from it. 

<code><pre>
Client                           Server
  | ---------- request ----------> |
  |                                |
  |                             execute
  |                                |
  | <---------  reply  ----------- |
  |                                |
 Read                              |
  |                                |
  | ---------- request ----------> |
  |                                |
  |                             execute
  |                                |
  | <---------  reply  ----------- |
</pre></code>

Multiple clients can be connected to the server at any given moment and make requests. This is called concurrency -  when two or more tasks can start, run and complete in overlapping periods. However, because of Redis' single-threadedness, concurrent requests are served one at a time and not in parallel. That means that if two clients send a request, even at the same time, one will always be processed and completed before the other begins.

<code><pre>
Client A                         Server                         Client B
  | ---------- request ----------> | <--------- request ----------- |
  |                             execute                             |
  | <---------  reply  ----------- |                                |
  |                             execute                             |
  |                                | ----------  reply  ----------> |
</pre></code>

While a single-threaded software design has its cons, among its pros is the fact that it is much simpler (to reason about, code, maintain...).

<sup>1</sup> some housekeeping tasks can be offloaded to child threads, e.g. persistence.

## Almost all Redis commands block

Because the Redis server uses a single thread for executing every command, every command blocks the server while it is being processed. Once a command's processing begins, and until its reply is written to the respective socket, the server is dedicated solely to its execution and does nothing but. Other commands (and their issuing clients) are held in bay, and will be attended to only once the server finishes with the current one.

This mode of operation endows several desirable traits to Redis' command execution. Borrowing from RDBMSs' ACID terminology, commands are:

* **Atomic** - "all or nothing", every command either runs completely or not
* **Consistent** - the data used by a command is valid and the command ensures consistency internally
* **Isolated** - totally, everything else is in waiting
* **Durability** - discussed earlier

Redis is able to serve many thousands of concurrent requests in a second. While each command blocks the server for its duration, each is usually completed in a few microseconds. Long-running commands can be identified with the [`SLOWLOG`](https://redis.io/commands/slowlog) command.

### Potentially long-running commands

Commands with O(N) time complexity can, depending on N's size (the volume of data), run for long times and block the server. For significant number of keys and large enough values, these include:

* Full iteration of the keyspace - [`KEYS`](https://redis.io/commands/keys), [`SAVE`](https://redis.io/commands/save)
* Big values: [`DEL`](https://redis.io/commands/del), [`DUMP`](https://redis.io/commands/dump), [`MIGRATE`](https://redis.io/commands/migrate), [`RESTORE`](https://redis.io/commands/restore)
* Big Hashes: [`HKEYS`](https://redis.io/commands/hkeys), [`HVALS`](https://redis.io/commands/hvals)
* Big Lists: [`LINDEX`](https://redis.io/commands/lindex), [`LINSERT`](https://redis.io/commands/linsert), [`LRANGE`](https://redis.io/commands/lrange), [`LREM`](https://redis.io/commands/lrem)
* Big Sets: [`SMEMBERS`](https://redis.io/commands/smembers), [`SDIFF`](https://redis.io/commands/sdiff), [`SINTER`](https://redis.io/commands/sinter), [`SUNION`](https://redis.io/commands/sunion) (and the respective Set operations `*STORE` variants)
* Big Sorted Sets: [`ZINTERSTORE`](https://redis.io/commands/zinterstore), [`ZUNIONSTORE`]((https://redis.io/commands/zunionstore), [`ZRANGE`](https://redis.io/commands/zrange), [`ZRANGEBYLEX`](https://redis.io/commands/zrangebylex), [`ZRANGEBYSCORE`](https://redis.io/commands/zrangebyscore) (and the respective `*REM*` and `*BY*` variants)

### Non-blocking commands

The following commands do not block the server or the client:

* Background saving - [`BGSAVE`](https://redis.io/commands/bgsave) & [`BGREWRITEAOF`](https://redis.io/commands/bgrewriteaof)
* Lazy deletion (v4) - [`UNLINK`](https://redis.io/commands/unlink)

### Client-blocking commands

The following commands do not block the server, only the client:

* Blocking PubSub commands - [`SUBSCRIBE`](https://redis.io/commands/subscribe) & [`SUBSCRIBE`](https://redis.io/commands/psubscribe)
* Blocking List operations - [`BLPOP`](https://redis.io/commands/blpop), [`BRPOP`](https://redis.io/commands/brpop) & [`BRPOPLPUSH`](https://redis.io/commands/brpoplpush)
* The [`MONITOR`](https://redis.io/commands/monitor) command
* The [`WAIT`](https://redis.io/commands/wait) command

## A late note about latency

Latency is a client-side measurement. It is defined as the period of time between sending a request and until getting the reply. It is largely made of the following parts:

* The time spent by the server being blocked by concurrent requests
* The network's round trip time, e.g. 1-2 milliseconds in a local network
* The execution time of the command, including processing the data and preparing the reply, e.g. a couple of microseconds in most cases but depends on complexity

Keeping latency to a minimum is in most cases a primary goal. Reducing the server's load is one way to improve (i.e. lower) latency, but there are also other ways to contribute to the overall effect. Communicating over the network is the most time-consuming element by orders of magnitude, and optimizing it can yield major gains.

## Variadic (a.k.a dynamic or indefinite arity) commands

Each command and its reply are communicated over the network. Network communication, compared to the time actually needed by the server for executing the command, is expensive in terms of latency.

That is where variadic commands come into play, which are **commands that accept one or more keys and/or arguments**. A good portion of Redis' commands do that, and by using this pattern instead of repeatedly calling the same command multiple times, latency is greatly reduced. This is mainly due the savings in network overehead - instead of communicating each command independently using its own packet(s), a single command is used in place. Another performance gain, albeit smaller, is due to the reduction of the server's context-switching efforts.

Because (nearly) every command blocks, variadic commands are no different and are atomic in terms of their execution. Once a command with multiple keys is executed, and until it completes, all other operations are blocked. 

Some Redis commands are variadic by nature, such as [`DEL`](https://redis.io/commands/del), whereas others have a special variadic variant such as [`GET`](https://redis.io/commands/get)'s multiple-flavored [`MGET`](https://redis.io/commands/mget). For example, consider the following `delOneByOne` function:

In [ ]:
from timeit import timeit
import redis

# example connection parameters 
config = {
    "host": "redis",
    "port": 6379
}

r = redis.StrictRedis(**config)

# Some definitions
count = 100
keys = ['key:{}'.format(i) for i in range(count)]

# A utility function for measuring and printing total execution time
def printTime(func):
    f = func.__name__ 
    t = timeit('{}()'.format(f), setup='from __main__ import {}'.format(f), number=1)
    print ('{}: {:.5f} seconds'.format(f, t))

# Note - this is an anti-pattern, do not use it
def setOneByOne():
    for i, key in enumerate(keys):
        r.set(key, i)

# Note - this is an anti-pattern, do not use it
def delOneByOne():
    for key in keys:
        r.delete(key)

setOneByOne()
printTime(delOneByOne)

Each iteration of the loops in `setOneByOne()` and  `delOneByOne` calls `set()` and `delete()`, respectively. A lot of time is wasted on sending the request and waiting for the reply. Compare the above with the following that uses the variadic, multi-key approach:

In [ ]:
def delVariadic():
    r.delete(*keys)

setOneByOne()
printTime(delVariadic)

The variadic invocation is done with a single command, so there's only one logical request and response involved in communicating it.

### Practical considerations for using variadic commands

The number of arguments that a variadic Redis command can accept is theoretically unlimited. Practically, up to several thousands is usually a sane limit, due to the following factors:

1. You do not want to block concurrent requests for too long
2. The configurable `client-output-buffer-limit normal` directive
3. A hard coded limit of 1GB for incoming queries (PROTO_MAX_QUERYBUF_LEN in server.h)

The payload's size also determines how many network packets will be needed to send it. For single-packet requests, the payload can be up to 1500 bytes usually.

### Exercise: using variadic commands

The following code gets keys using the non-variadic `getOneByOne()` command (this is only an example, so the reply is discarded):

In [ ]:
# Note - this is an anti-pattern, do not use it
def getOneByOne():
    for key in keys:
        _ = r.get(key)

setOneByOne()
printTime(getOneByOne)

Convert it to use the variadic multi-key `mget()`:

In [ ]:
def getVariadic():
   r.mget(*keys)

printTime(getVariadic)

## Pipelines

Pipelining is an extremely effective way to reduce the overall latency and resource consumption. Similar to batching, pipelining groups together multiple commands and sends them to the server using minimal network resources (compared to communicating them one by one). The responses are, similarly, returned to the client in bulk once the pipeline is executed in whole.

Pipelines are implemented on both sides of the communication channel: the client needs to "know" how to buffer pipelined requests, and the server knows how to execute the entire pipeline without waiting for the client to read each reply. 

### When to use pipelines

Pipelining's contribution to latency, much like variadic commands, is in reducing the total number of network requests and thus saving on the round trip times. Pipelining should be used whenever multiple commands can be aggregated by the application, and as long as commands are not dependent on each other.

A pipeline can contain any number of Redis commands, and any command can be in a pipeline (excluding the blocking PubSub and List client commands). A variadic command is preferable to a pipeline, but pipelines are useful when:

1. There is no variadic variant for a command
2. Multiple, potentially different, commands can be sent together

### How to use pipelines

When using the `redis-py` client, pipelines are treated just like regular connections. Once the pipeline is ready for execution, call its `execute()` function. 

For demonstration purposes, we'll use the [`SET`](https://redis.io/commands/set) command (although it does have a variadic counterpart, the [`MSET`](https://redis.io/commands/mset) command). Recall the `setOneByOne()`  implementation - the same payload can be sent using a single pipeline:

In [ ]:
def pipelinedSets():
    p = r.pipeline(transaction=False)
    for i, key in enumerate(keys):
        p.set(key, i)
    p.execute()

delVariadic()
printTime(setOneByOne)
delVariadic()
printTime(pipelinedSets)

The pipeline buffers the requests, in this case the calls to `set()` from the loop, and sends them together to the server when it is executed.

### Pipelining means replies only come later

It is easy to forget that replies to pipelined commands are sent back only **after** the pipeline has been sent for execution.

In [ ]:
p = r.pipeline(transaction=False)
for key in keys[:3]:
    print (p.get(key))

values = p.execute()
print (values)

This means that trying to read replies before  `execute()`  is called is meaningless. That means that you should **avoid the following anti-pattern** that tries to use the reply from a pipelined command inside the same pipeline:

In [ ]:
# Beware of premature use of pipelined replies
p = r.pipeline()
val = p.incrby('somecounter', 1)  # p is a pipeline object!
p.set('somekey', val)             # this sets the value of `somekey` to the pipeline's "pointer"
p.execute()

print (r.get('somekey'))

### Pipelines are not atomic

When pipelining is used, the client-side experience is that of bulk processing - batches (pipelines) are sent to the server, whereupon the batch's replies are returned, possibly so that another batch can be sent.

This is slightly different from the server's perspective. While the pipeline is received and replied to in whole, serving only one pipeline at a time means impacting the overall average latency. Consider as an example a scenario in which one client sends a pipeline with a 100 commands at the same time as 100 different clients send each just one. If the server was to serve the single pipeline from beginning to end, the average latency all commands processed after it would be impacted.

To avoid "starving" other clients' requests and improve concurrency, the Redis server may interleave the execution of pipelined operations from one client with those from others (whether these are pipelined or not). This means that while the order of execution of a pipeline's commands is guaranteed, data may change between one operation to another by another client.

### Practical considerations for using pipelining

Any Redis command can be a part of a pipeline as long is it does not block the client. Pipelines can have any arbitrary size. That said, buffering the pipelines at both ends requires RAM. Instead of pipelining entire workloads, chunk them into sanely-sized batches up to a couple of thousands of commands.

To determine the optimal size for your pipeline - **experiment!**

In [ ]:
# Send ping in pipelines of up to `size`
def pipelinedPings(count, size):
    p = r.pipeline(transaction=False)
    for i in range(count):
        _ = p.ping()
        if i % size == 0:
            p.execute()
            p = r.pipeline()
    p.execute()

sizes = [2**i for i in range(2,14)]
setup = 'from __main__ import pipelinedPings'
for s in sizes:
    t = timeit('pipelinedPings(10000,{})'.format(s), setup=setup, number=1)
    print ('Pipeline size: {}, time: {:.5f} seconds'.format(s, t))

### Exercise: using pipelines

Recall the anti-pattern of `getOneByOne()` command and convert it to use a pipelined `get()`s instead:

In [ ]:
def pipelinedGets():
    p = r.pipeline(transaction=False)
    for key in keys:
        _ = r.get(key)
    p.execute()
    
printTime(pipelinedGets)

## Transactions

Redis is (mostly) single-threaded, so every command is executed atomically and is fully isolated from all others. When serving multiple client connections, however, Redis interleaves the incoming requests for concurrency. A transaction provides the means to ensure that a block of commands is executed without the possibility of data being modified by other clients. Put differently, Redis transactions are atomic and block the server while they are run.

### Transaction semantics and mechanics

A Redis transaction, also known as a `MULTI/EXEC` block, is made of multiple commands (excluding, of course, the client-blocking ones). A transaction's commands are preceded by the command [`MULTI`](https://redis.io/commands/multi) , and the transaction is executed with the [`EXEC`](https://redis.io/commands/exec) command. Alternatively, a transaction may be discarded before it is `EXEC`uted with the [`DISCARD`](https://redis.io/commands/discard) command.

Transactions may be pipelined, although this is not mandatory. Similarly to the behavior of pipelines, the transaction's commands are buffered until execution and responses, if any, are returned to the client only after the fact.

A transaction is atomic in the sense that once its execution begins, the commands in it will be run one after the other (each being atomic by its own right), without   operations being processed between them. That said, Redis' transaction atomicity does not mean "all or nothing" - a transaction that stops halfway due to an error will leave the data in inconsistent state. There is no rollback of transactions, whether completed or aborted.

### How to transact

The redis-py client implements support for transactions with the pipeline object, when the latter is created with default  `transaction=True` argument. Once the transaction is created, it is used in a fashion identical to a regular (non-transactional) pipeline.

As an example consider a database used for managing account balances and transfers. Let us assume that the details of each account are stored using a Redis Hash, for example:

In [ ]:
def createAccount(accountid, balance):
    key = 'account:{}'.format(accountid)
    dic = { 'accountid': accountid, 'status': 'active', 'balance': balance}
    return r.hmset(key, dic)

# Create two accounts, one with nothing and the other some
print (createAccount(1, 0))
print (createAccount(2, 10))

The next task would be to implement a mechanism for transferring an amount from one account's balance to another. To prevent other clients from using the data while it is only partially update, the transfer's logic is wrapped in a Redis transaction:

In [ ]:
def transferAmount(debit, credit, amount):
    amount = float(amount)
    debitkey = 'account:{}'.format(debit)
    creditkey = 'account:{}'.format(credit)

    tx = r.pipeline()  # same as `r.pipeline(transaction=True)`
    tx.hincrbyfloat(debitkey, 'balance', -amount)
    tx.hincrbyfloat(creditkey, 'balance', amount)
    return tx.execute()

print (transferAmount(2, 1, 1))

### Optimistic concurrency control

While the use of a transaction ensures its atomicity, all replies from the server are sent back only after it was executed. Just like with non-transactional pipelines, this means that any data used inside transaction must be provided outside it. Put differently, commands in a transaction can't use the replies from other commands preceding them.

However, in many cases a transaction relies on data already stored in the database to perform more modifications. Continuing the accounting example from above, it would make sense to verify that the debited account has sufficient funds prior to making the transfer. Before modifying the data, we need to check whether the current account balance allows that. 

Because the current balance can not be checked inside the transaction, it is tempting (**but wrong**) to use an implementation such as:

In [ ]:
def incorrectCheckBalanceAndTransferAmount(debit, credit, amount):
    amount = float(amount)
    debitkey = 'account:{}'.format(debit)
    creditkey = 'account:{}'.format(credit)
    fname = 'balance'

    balance = r.hget(debitkey, fname)
    # Potential race condition - start
    if balance >= amount: 
        tx = r.pipeline()
        tx.hincrbyfloat(debitkey, fname, -amount)
        tx.hincrbyfloat(creditkey, fname, amount)
        return tx.execute()
    # Potential race condition - end
    else:
        raise Exception('insufficient funds - go and earn some')

The seemingly innocent code above hides a race condition that while perhaps rare, is nonetheless less than desirable. The problem lies in the period between between getting `hget()`'s reply in the client and the server running the transaction after receiving `execute()`. Given the right circumstances and during that period, another client can modify the debited account's balance and thus violate a business requirement.

Locking, or optimistic concurrency control, is one way for solving this. Redis, however, does not offer any means for locking data. Instead, it encourages you to use optimistic concurrency control. With optimistic concurrency control, transactions assume that the data they rely on remains unchanged throughout their preparation and until they are executed. Changes to the data will cause an optimistic transaction to fails. Put differently, an optimistic transaction fails if the data it relies on changes.

Redis provides a means to detect changes in the data before the transaction is executed by the server via the use of the [`WATCH`](https://redis.io/commands/watch) command. `WATCH`ing a value has two effects:

1. The following commands in the will be executed immediately until the `MULTI` command is used
2. The transaction will fail if the watched value had changed since it was last read. It is up to the application to catch this failure and retry the transaction, if needed.

The following is an example that uses optimistic concurrency control for checking the balance and making a transfer:

In [ ]:
def checkBalanceAndTransferAmount(debit, credit, amount):
    amount = float(amount)
    debitkey = 'account:{}'.format(debit)
    creditkey = 'account:{}'.format(credit)
    fname = 'balance'
    
    while True:
        try:
            tx = r.pipeline()
            tx.watch(debitkey)
            balance = float(tx.hget(debitkey, fname))
            tx.multi()
            if balance >= amount:
                tx.hincrbyfloat(debitkey, fname, -amount)
                tx.hincrbyfloat(creditkey, fname, amount)
                return tx.execute()
            else:
                raise Exception('insufficent funds - time to get a job')
        except WatchError:
            # Reaching here means that the watched 'balance' value had changed,
            # so we can just retry or use any other backoff logic
            continue

### Exercise: using transactions

Implement the `mergeAccounts(src, dst)` function that performs the following logic in an optimistic transactional manner:

1. Sets the value of the `status` field of the `source` account Hash key to "merged into account {destination}"
2. Increments the `balance` of the `destination` account with the `source`'s  balance
3. Sets the `balance` of the `source` account to 0

In [ ]:
def mergeAccounts(src, dest):
    amount = float(amount)
    srckey = 'account:{}'.format(src)
    dstkey = 'account:{}'.format(dst)
    
    # Your answer here
    pass

## Further reading

* [Request/Response protocols and RTT](https://redis.io/topics/pipelining)
* [Transactions](https://redis.io/topics/transactions)